### COM361 &mdash; Introdução a Otimização &mdash; 2024, Prof. Amit ###

# Controle de Fake News #

#### Wagner Franco da Silva Junior (wagnerjunior@poli.ufrj.br)

*****

### Índice

1. [Introdução](#1.-Introdução)
1. [Modelo ISR](#2.-Modelo-ISR)
1. [Função Objetivo](#3.A.-Função-Objetivo)
1. [Controle Ótimo](#4.-Controle-Ótimo)
1. [Restrições Orçamentárias](#5.-Restrições-Orçamentárias)
1. [Controle em Tempo Real](#6.-Controle-Em-Tempo-Real)
1. [Incertezas](#7-Incertezas)
1. [Conclusão](#8.-Conclusão)

## 1. Introdução ##

O trabalho tem por objetivo realizar o controle ótimo de um problema de epidemiologia social, que são as fake news. Atualmente com o avanço da tecnologia, as fake news se tornaram um problema cada vez mais grave principalmente para pessoas mais leigas com relação aos perigos que a internet pode proporcionar, sendo alvos fáceis. A partir disso, foi fornecido um modelo discreto do problema onde será feita a otimização:

$$ x_1(k + 1) = x_1(k) - \beta x_1(k)x_2(k) - bu(k)x_1(k) $$
$$ x_2(k + 1) = x_2(k) + \beta x_1(k)x_2(k) - \gamma x_2(k)^2 $$

$x_1(k)$ representa os ignorantes/indecisos 

$x_2(k)$ representa os espalhadores de fake news 

$u(k)$ representa o nosso controle, que pode ser visto como "vacinas" de educação 

A partir dessas informações e através do modelo ISR, será proposta uma função objetivo que se adeque ao sistema e feito o controle ótimo e um controle em tempo real.

## 2. Modelo ISR ##

O modelo ISR (Ignorant - Spreader - Recovered) para propagação de fake news se baseia em algumas premissas para seu funcionamento, que são as seguintes:

I + S = 2S (Um propagador consegue converter um ignorante a outro propagador)

S + S = S + R (Dois propagadores ao se encontrarem, um deles percebe que a informação já é bem conhecida e "desanima" de espalhar)

S + R = 2R (Um recuperado consegue converter um propagador)

As taxas de conversão são justamente as constantes estabelecidas no modelo, $\beta$ é a taxa de conversão de um ignorante para propagador e $\gamma$ é a taxa de um propagador para recuperado.

## 3. Função Objetivo ##

Para a escolha da função objetivo foi feito uma análise da discussão da página 223 onde é introduzido o conceito de reprodução básica, que diz que o número mínimo que a informação vai continuar se espalhando é de $R_0$ = $\dfrac{\gamma}{\beta}$, e a partir disso sabemos que a taxa de aumento nos propagadores precisa ser menor que esse valor, assim temos que:

$$ x_1(k) + x_2(k) < \dfrac{\gamma}{\beta} $$

Dessa forma, tendo em vista que o objetivo central de toda a otimização é minimizar o número de propagadores, a função objetivo precisa minimizar tanto o custo de controle (de forma a evitar esforços desnecessários) e minimizar a diferença $x_1(k) + x_2(k) - \dfrac{\gamma}{\beta}$ para garantir que o valor encontrado seja de fato ótimo.

Assim, basta que integremos no horizonte a seguinte função: 

$$
J = \int_{0}^{T_f} \left[ R \cdot u(t)^2 + Q \cdot \left( x_1(t) + x_2(t) - \frac{\gamma}{\beta} \right)^2 \right] dt,
$$
onde:
- \(R\): penaliza o esforço de controle $u(t)$.
- \(Q\): penaliza o desvio do estado $(x_1(t) + x_2(t)$ em relação ao objetivo $\frac{\gamma}{\beta}$.
- $u(t)$: controle.


## 4. Controle Ótimo ##

In [ ]:
using JuMP
using Ipopt 
using PyPlot

function solveISR(β, γ, b, T, x1_init, x2_init, R, Q)
    k = length(T)
    T_max = T[end]  
    xw = [γ / β; γ / β]

    m = Model(Ipopt.Optimizer)

    @variable(m, x1[1:T_max] >= 0)
    @variable(m, x2[1:T_max] >= 0)
    @variable(m, u[1:T_max] >= 0)

    @constraint(m, x1[1] == x1_init)
    @constraint(m, x2[1] == x2_init)

    for t in 1:T_max - 1
        @constraint(m, x1[t + 1] == x1[t] - β * x1[t] * x2[t] - b * u[t] * x1[t])
        @constraint(m, x2[t + 1] == x2[t] + β * x1[t] * x2[t] - γ * x2[t]^2)
    end

    @objective(m, Min, R * sum(u .^ 2) + Q * sum((x1[T] .+ x2[T] .- xw[1]) .^ 2))

    optimize!(m)

    x1_opt = value.(x1)
    x2_opt = value.(x2)
    u_opt = value.(u)
    J1 = R * sum(u_opt .^ 2)
    J2 = Q * sum((x1_opt[T] .+ x2_opt[T] .- xw[1]) .^ 2)
    return (J1, J2, x1_opt, x2_opt, u_opt)
end

β = 0.4                
γ = 0.1               
b = 0.05                 
T = [1, 20, 50, 60, 80, 100, 120]      
x1_init = 1.0        
x2_init = 0.5          
R = 0.001            
Q = 1.0 

(J1, J2, x1_opt, x2_opt, u_opt) = solveISR(β, γ, b, T, x1_init, x2_init, R, Q)

figure(figsize=(12, 4))
plot(1:length(x1_opt), x1_opt, "b-", label="Ignorantes (x1)")
plot(1:length(x2_opt), x2_opt, "r-", label="Espalhadores (x2)")
plot(1:length(u_opt), u_opt, "g-", label="Controle (u)")
legend()
xlabel("Tempo")
ylabel("Valores")
title("Controle Ótimo")
grid(true)

## 5. Restrições Orçamentárias ##

Nesse próximo script, houve uma adição de restrição orçamentária para o valor que pode ser gasto em $u(k)$ de forma que simule uma situação real onde o controle que podemos aplicar não consegue ser infinito.

$$ u(k) \le U_{max}$$

Os valores de todos os parâmetros são exatamente os mesmo, porém pela saída podemos perceber que o controle agora tem um pico menor de altura ao ser impactado pela restrição

In [ ]:
using JuMP
using Ipopt
using PyPlot

function solveISR(β, γ, b, T, x1_init, x2_init, R, Q, U_max)
    k = length(T)          
    T_max = T[end]     
    xw = [γ / β; γ / β] 

    m = Model(Ipopt.Optimizer)

    @variable(m, x1[1:T_max] >= 0)
    @variable(m, x2[1:T_max] >= 0)
    @variable(m, u[1:T_max] >= 0) 

    @constraint(m, x1[1] == x1_init)
    @constraint(m, x2[1] == x2_init)

    for t in 1:T_max - 1
        @constraint(m, x1[t + 1] == x1[t] - β * x1[t] * x2[t] - b * u[t] * x1[t])
        @constraint(m, x2[t + 1] == x2[t] + β * x1[t] * x2[t] - γ * x2[t]^2)
    end

    @constraint(m, sum(u) <= U_max)

    @objective(m, Min, R * sum(u .^ 2) + Q * sum((x1[T] .+ x2[T] .- xw[1]) .^ 2))

    optimize!(m)

    x1_opt = value.(x1)
    x2_opt = value.(x2)
    u_opt = value.(u)
    J1 = R * sum(u_opt .^ 2)
    J2 = Q * sum((x1_opt[T] .+ x2_opt[T] .- xw[1]) .^ 2)
    return (J1, J2, x1_opt, x2_opt, u_opt)
end

β = 0.4      
γ = 0.1            
b = 0.05               
T = [1, 20, 50, 60]     
x1_init = 1.0           
x2_init = 0.5              
R = 0.001             
Q = 1.0                
U_max = 2             

(J1, J2, x1_opt, x2_opt, u_opt) = solveISR(β, γ, b, T, x1_init, x2_init, R, Q, U_max)

figure(figsize=(12, 4))
plot(1:length(x1_opt), x1_opt, "b-", label="Ignorantes (x1)")
plot(1:length(x2_opt), x2_opt, "r-", label="Espalhadores (x2)")
plot(1:length(u_opt), u_opt, "g-", label="Controle (u)")
legend()
xlabel("Tempo")
ylabel("Valores")
title("Controle com Restrição Orçamentária")
grid(true)

## 6. Controle em Tempo Real ##

A ideia consiste, basicamente, em fazer um controle de predição a cada instante de tempo estabelecido, sempre levando como referência o valor no horizonte para os próximos $T_f$ momentos, de forma que a cada instante que passa, se repete o mesmo procedimento.

In [ ]:
using JuMP
using Ipopt
using PyPlot

function solveISRStep(β, γ, b, x1_prev, x2_prev, xw, R, Q, U_max, horizon)
    m = Model(Ipopt.Optimizer)

    @variable(m, x1[1:horizon + 1] >= 0)
    @variable(m, x2[1:horizon + 1] >= 0) 
    @variable(m, u[1:horizon] >= 0) 

    @constraint(m, x1[1] == x1_prev)
    @constraint(m, x2[1] == x2_prev)

    for t in 1:horizon
        @constraint(m, x1[t + 1] == x1[t] - β * x1[t] * x2[t] - b * u[t] * x1[t])
        @constraint(m, x2[t + 1] == x2[t] + β * x1[t] * x2[t] - γ * x2[t]^2)
    end

    @constraint(m, sum(u) <= U_max)

    @objective(m, Min, R * sum(u .^ 2) + Q * sum((x1[end] + x2[end] - xw[1])^2))

    optimize!(m)

    u_opt = value(u[1])

    return u_opt
end

function controlISRRealTime(β, γ, b, T, x1_init, x2_init, R, Q, U_max, horizon)
    T_max = T[end]          
    xw = [γ / β; γ / β]

    x1_opt = [x1_init]
    x2_opt = [x2_init]
    u_opt = []

    for t in 1:T_max
        u_t = solveISRStep(β, γ, b, x1_opt[end], x2_opt[end], xw, R, Q, U_max, horizon)

        push!(u_opt, u_t)

        x1_next = x1_opt[end] - β * x1_opt[end] * x2_opt[end] - b * u_t * x1_opt[end]
        x2_next = x2_opt[end] + β * x1_opt[end] * x2_opt[end] - γ * x2_opt[end]^2

        push!(x1_opt, x1_next)
        push!(x2_opt, x2_next)

        if abs(x1_next + x2_next - xw[1]) < 1e-3
            break
        end
    end

    return x1_opt, x2_opt, u_opt
end

β = 0.4             
γ = 0.1                
b = 0.05               
T = 1:60             
x1_init = 1.0        
x2_init = 0.5  
R = 0.001            
Q = 1.0               
U_max = 2 
horizon = 5

x1_opt, x2_opt, u_opt = controlISRRealTime(β, γ, b, T, x1_init, x2_init, R, Q, U_max, horizon)

figure(figsize=(12, 4))
plot(1:length(x1_opt), x1_opt, "b-", label="Ignorantes (x1)")
plot(1:length(x2_opt), x2_opt, "r-", label="Espalhadores (x2)")
plot(1:length(u_opt), u_opt, "g-", label="Controle (u)")
legend()
xlabel("Tempo")
ylabel("Valores")
title("Controle em Tempo Real")
grid(true)

## 7. Incertezas ##

Incertezas nos parâmetros ($\beta , \gamma, x_1, x_2$):
- O controle ótimo normal não lida bem com incertezas pois assume-se que tanto os parâmetros quanto os estados são perfeitamente conhecidos, se os valores reais forem diferentes dos utilizados no modelo, o controle pode ser inviável e completamente inadequado. 
- O controle preditivo em tempo real lida melhor com as incertezas pois a cada passo o controle é recalculado com base no estado atual, dessa forma pode ser reajustado dinamicamente caso aja uma alteração nos parâmetros, da mesma forma que está continuamente medindo os estados $x_1 e x_2$ em tempo real.


## 5. Conclusão ##

Pelos resultados encontrados, podemos visualizar graficamente o funcionamento do modelo ISR, de forma que conforme os ignorantes vão diminuindo, conforme o tempo passa e vão tendo os encontros I + S, e os espalhadores em certo momento aumentam com a diminuição dos ignorantes que se tornam espalhadores e com o tempo vão tendendo a zero a partir do momento que não há mais ignorantes. O controle também se mostrou com um comportamento coerente, sempre que aumento a relação entre ignorantes x espalhadores também há um aumento na ação de controle e a partir disso diminui mais drasticamente os ignorantes. A ação de controle também se mostrou afetada pela limitação de orçamento, de forma que no script com a restrição a ação é bem menor, mesmo estando com os mesmos parâmetros de antes. Para próximos passos, poderia ser adicionado uma nova restrição com a relação entre recuperados x espalhadores, como no modelo padrão do ISR, para se observar o comportamento completo do sistema.  